In [1]:
%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path 
current_path = Path().resolve()
abs_path = str(current_path.parent)
sys.path.append(abs_path)

RAW_PATH = current_path.parent / 'data' / 'raw'
OUTPUT_PATH = current_path.parent / 'data' / 'output'

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_columns', None)

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.offline as offline
offline.init_notebook_mode(connected=True)


In [2]:
import scripts.example

hi


In [5]:
from h3 import h3


In [7]:
import folium
m = folium.Map(
    location=[37.3615593, -122.0553238],
    zoom_start=13
)

for i in range(1, 15): 
    h3_address = h3.geo_to_h3(37.3615593, -122.0553238, i)
    folium.Polygon(
        h3.h3_to_geo_boundary(h3_address),
        tooltip=i
    ).add_to(m)

m


In [50]:
from pyathena import connect

conn = connect(s3_staging_dir='s3://aws-athena-queries-result-east-2/join-pems-waze',
               region_name='us-east-2')

In [107]:
query = """SELECT 
            ST_GEOMETRY_TO_TEXT(ST_BUFFER(ps.point, 0.001)) as b_001,
            ST_GEOMETRY_TO_TEXT(ST_BUFFER(ps.point, 0.005)) as b_005,
            ST_GEOMETRY_TO_TEXT(ST_BUFFER(ps.point, 0.01)) as b_01,
            ST_GEOMETRY_TO_TEXT(ST_BUFFER(ps.point, 0.02)) as b_02,
            ST_GEOMETRY_TO_TEXT(ST_BUFFER(ps.point, 0.03)) as b_03,
            ST_GEOMETRY_TO_TEXT(ST_BUFFER(ps.point, 0.04)) as b_04,            
            ST_GEOMETRY_TO_TEXT(ST_BUFFER(ps.point, 0.05)) as b_05,
            ST_GEOMETRY_TO_TEXT(ST_BUFFER(ps.point, 0.1)) as b_1,
            ST_GEOMETRY_TO_TEXT(ST_BUFFER(ps.point, 0.5)) as b_5
            FROM (select * from pems.pems_station_unique limit 1) ps"""

In [108]:
data = pd.read_sql_query(query, conn)

In [109]:
data = data.T

In [110]:
def to_poly(x):
    
    x = x.replace('POLYGON', '').replace('(', '[').replace(')', ']').replace(', ', '],[').replace(' ', ',')[1:]
    x = ast.literal_eval(x)
    x = map(lambda x: [x[1], x[0]], x)
    return list(x)

In [111]:
data['poly'] = data[0].apply(to_poly)

In [112]:
m = folium.Map(
    location=[38.413138000000004, -121.4834360000000],
    zoom_start=13
)

for i, row in data.iterrows():

    folium.Polygon(
        row['poly'],
        tooltip=i
    ).add_to(m)

m

In [34]:
h3_address = h3.geo_to_h3(37.3615593, -122.0553238, 5) # lat, lng, hex resolution
hex_center_coordinates = h3.h3_to_geo(h3_address) # array of [lat, lng]

In [35]:
hex_center_coordinates

[37.34579337536847, -121.97637597255115]

[[37.27135586673191, -121.91508032705622],
 [37.353926450852256, -121.86222328902491],
 [37.42834118609436, -121.92354999630152],
 [37.42012867767779, -122.0377349642703],
 [37.33755608435299, -122.09042892904398],
 [37.26319797461824, -122.02910130919]]

In [25]:
geoJson = {'type': 'Polygon',
 'coordinates': [[[37.813318999983238, -122.4089866999972145],
 [ 37.7866302000007224, -122.3805436999997056 ],
 [37.7198061999978478, -122.3544736999993603],
 [ 37.7076131999975672, -122.5123436999983966 ],
 [37.7835871999971715, -122.5247187000021967],
 [37.8151571999998453, -122.4798767000009008]]] }
hexagons = list(map(lambda x: (x, h3.h3_to_geo_boundary(x)), h3.polyfill(geoJson, 8)))

In [27]:
pd.DataFrame(hexagons, columns=['h3_address', 'boundary'])

,h3_address,boundary
0,88283095edfffff,"[[37.736924821938466, -122.45706587893915], [3..."
1,8828309517fffff,"[[37.7286872728031, -122.46008632427026], [37...."
2,882830828dfffff,"[[37.76298184766419, -122.4094546324225], [37...."
3,8828308289fffff,"[[37.7605872270941, -122.41984858577302], [37...."
4,8828309521fffff,"[[37.724190882360375, -122.41418913254694], [3..."
5,88283082cbfffff,"[[37.74411416961636, -122.42589766815368], [37..."
6,8828308763fffff,"[[37.8062480134132, -122.45065030990423], [37...."
7,88283095c9fffff,"[[37.721490423390215, -122.49124063086623], [3..."
8,8828308291fffff,"[[37.778105905454524, -122.44195558308803], [3..."
9,882830951dfffff,"[[37.71460844120174, -122.45574066521505], [37..."
